#I. The model works as follows:
Go to the last cell to see outputs.
1. Train the backward model to generate backward sentences, starting from a given word. 
2. Train the forward model to generate backward sentences, starting from a given sequence. (collected during inference of backward model)
3. For inference using joint beam search for decoding, choose 2 pseudo words from given pun dataset (eg. "countv01", "countv08"), and input it to encoder_bw, collect the hidden and cell state and input it to decoder_bw.JointBeamSearch(mode=eval) 
4. Reverse the generated backward sequence and forward the "sequence" to the forward models in similar way(encoder_fw, decoder_fw)

5. Concatenate the forward and backward sequences(replace the split word with target word), and disambiguate

In [ ]:
from google.colab  import drive
drive.mount('/content/gdrive')
!ls

Mounted at /content/gdrive
gdrive	sample_data


In [ ]:
#@title
path = '/content/gdrive/MyDrive/IE-643/enwiki-latest-pages-articles.xml.bz2'
%cd gdrive
%cd MyDrive
%cd IE-643
!ls

/content/gdrive
/content/gdrive/MyDrive
/content/gdrive/MyDrive/IE-643
 changed.txt
 Corpusdump1.pickle
 Corpusdump2.pickle
 Corpusdump.pickle
 Corpusdump.txt
 corpus.txt
 data.json
 dump1.pickle
 dump2.pickle
 dumpfile.bz2
 dump.pickle
 enwiki-latest-pages-articles.xml.bz2
 filename.pickle
 filename.pkl
 file.pickle
 glove.6B.50d.txt
 glove.6B.zip
 glovefile
 out.csv
 out.gsheet
 out.pickle
 output0.csv
 output10.csv
 output11.csv
 output12.csv
 output13.csv
 output14.csv
 output15.csv
 output16.csv
 output17.csv
 output18.csv
 output19.csv
 output1.csv
 output2.csv
 output3.csv
 output4.csv
 output5.csv
 output6.csv
 output7.csv
 output8.csv
 output9.csv
 pundata.csv
 pundata.gsheet
 Pun-GAN
 Pun-Generation
 save.csv
 save.pickle
 save.pik
 save.txt
 semcor3.0.tar.gz
 semcor.py
 semeval2017_task7
 semeval2017_task7.tar.xz
'semeval2017_task7.tar.xz (Unzipped Files)'
 tagged.pickle
 tagged.pos
 text
 wikiextractor


In [ ]:
#@title
!pip install wikiextractor

In [ ]:
#@title
! wikiextractor '/content/gdrive/MyDrive/IE-643/enwiki-latest-pages-articles.xml.bz2'  #extracting and cleaning the dumpfile for pre-processing

/bin/bash: wikiextractor: command not found


In [ ]:

import nltk

nltk.download('all')
import os
import glob
import re
from nltk import word_tokenize
from nltk import sent_tokenize
!pip install pywsd
#polysemes in homographic pun dataset
!pip install -U wn==0.0.23 #downgrading for pywsd(mentioned in the paper) to work
from nltk.corpus import wordnet as wn
from pywsd import disambiguate
from pywsd.similarity import max_similarity as maxsim

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

Warming up PyWSD (takes ~10 secs)... took 4.550925254821777 secs.


Pre-processing steps:
1. Corpus texts firstly lowercased and
tokenized, and all numeric characters are replaced
with “#”
2. We split the texts into sentences and
discard the sentences whose length is less than 5
words or more than 50 words
3. We then select polysemes appearing in the homographic pun data set
(Miller et al., 2017) and pun websites
4. Those polysemes in the corpus are replaced by the labeled
sense. We restrict that each sentence can be labeled with at most two polysemes in order to train
a reliable language model
5. If there are more polysemes in one sentence, we keep the last two because in our observation we found pun words tend
to occur near the end of a sentence.
6. After labeling, we keep the 105,000 most frequently occurring words and other words are replaced with the
'unk>' token. 
6.  We discard the sentences with
two or more 'unk>' tokens. There are totally
3,974 distinct labeled senses corresponding to a
total of 772 distinct polysemes.
7. While training the language model we use
2,595,435 sentences as the training set, and
741,551 sentences as the development set to decide when to stop training
8.  Since we hope to
generate a sentence containing the specified word
sense, we treat different senses of the same word
as independent new pseudo-words. We label the
senses of words with Word Sense Disambiguation (WSD) tools, and then we train the language
model using the corpus with labeled senses so
that for each word sense we can generate a sentence accordingly. We use the Python Implementations of WSD Technologies1 for WSD. This tool
can return the most possible sense for the target
word based on WordNet (Miller, 1995). We attach the sense label to the word and form a new
pseudo-word accordingly.

For training, first each word in the corpus is tagged with one word sense. Joint model is applicable during inference

In [ ]:
#@title
path = '/content/gdrive/MyDrive/IE-643/Pun-Generation/sem17_task7/test/subtask2-homographic-test.xml'
path2 = '/content/gdrive/MyDrive/IE-643/Pun-Generation/sem17_task7/test/subtask2-homographic-test.gold'


About the dataset

Semeval 2017-task7-subtask1

The dataset contains 2250 puns, where each word is annotated with 1 or 0, indicating a pun, or not pun

TODO- Extract two words and their senses


In [ ]:
import pandas as pd

pundata = pd.read_csv('/content/gdrive/MyDrive/IE-643/Pun-Generation/sem17_task7/test/pundata - pundata.csv')
pundata

,Unnamed: 0,word,id,sense1,sense2
0,0,saving,hom_2_9,save%2:40:02::,save%2:41:02::
1,1,sting,hom_3_7,sting%1:26:01::,sting%1:04:01::
2,2,entrenched,hom_4_5,entrenched%5:00:00:invulnerable:00,entrenched%5:00:00:established:00
3,3,forge,hom_5_15,forge%2:36:00::,forge%2:38:01::
4,4,hit,hom_7_14,hit%1:04:00::,hit%1:04:02::
...,...,...,...,...,...
1293,1293,waste,hom_2244_9,waste%1:27:00::,waste%2:29:00::
1294,1294,battery,hom_2245_10,battery%1:06:00::,battery%1:14:01::
1295,1295,played,hom_2248_8,play_out%2:40:00::,play_out%2:36:00::
1296,1296,bright,hom_2249_13,bright%3:00:00::,bright%5:00:00:intelligent:00


In [ ]:
import glob
import re
m = '0'
n ='0'
pseudo1  = '0'
pseudo2 = '0'
def get_synset_from_id(word, sense1, sense2):
  global m, n, pseudo1, pseudo2
  x = wn.synsets(word)
  for i in range(len(x)):
    lemmas = x[i].lemmas()
    for j in range(len(lemmas)):
      if lemmas[j].key() == sense1:
        m = x[i]
        pseudo1 = str(m)
        pseudo1 = re.sub('[()]', '', pseudo1)
        pseudo1 = re.sub('[.]', '', pseudo1)
        pseudo1 = re.sub('Synset', '', pseudo1)
        pseudo1 = re.sub("[']", "", pseudo1)
      if lemmas[j].key() == sense2:
        n = x[i]
        pseudo2 = str(n)
        pseudo2 = re.sub('[()]', '', pseudo2)
        pseudo2 = re.sub('[.]', '', pseudo2)
        pseudo2 = re.sub('Synset', '', pseudo2)
        pseudo2 = re.sub("[']", "", pseudo2)
  return (pseudo1, m), (pseudo2, n)



In [ ]:
i=0
pun = {}
for index, row in pundata.iterrows():
  word = row['word']
  pun[word] = []
for index, row in pundata.iterrows():
  word = row['word']
  sense1 = row['sense1']
  sense2 = row['sense2']
  A= get_synset_from_id(word, sense1, sense2)
  
  pun[word].append(A) 
  i = i+1
len(pun)  #970 pun words in all Each pun word has 2(or more than 2 labelled senses)

970

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
path = '/content/gdrive/MyDrive/IE-643/text'
path1 = '/content/gdrive/MyDrive/IE-643/corpus.txt'

import os
import glob

files = []
for filename in os.listdir(path):
  files.append(filename)
files

files1 = {}
with open(path1, 'a') as file_object:
  for filename in files:
    files1[filename] = []
    for file in os.listdir(os.path.join(path, filename)):
      with open(os.path.join(path, filename, file), 'rU') as f:
        textfile = f.read()
        files1[filename].append(file)
print(files)
print(files1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: 'U' mode is deprecated


['AA', 'AB', 'AC', 'AD']
{'AA': ['wiki_00', 'wiki_01', 'wiki_02', 'wiki_03', 'wiki_04', 'wiki_05', 'wiki_06', 'wiki_07', 'wiki_08', 'wiki_09', 'wiki_10', 'wiki_11', 'wiki_12', 'wiki_13', 'wiki_14', 'wiki_15', 'wiki_16', 'wiki_17', 'wiki_18', 'wiki_19', 'wiki_20', 'wiki_21', 'wiki_22', 'wiki_23', 'wiki_24', 'wiki_25', 'wiki_26', 'wiki_27', 'wiki_28', 'wiki_29', 'wiki_30', 'wiki_31', 'wiki_32', 'wiki_33', 'wiki_34', 'wiki_35', 'wiki_36', 'wiki_37', 'wiki_38', 'wiki_39', 'wiki_40', 'wiki_41', 'wiki_42', 'wiki_43', 'wiki_44', 'wiki_45', 'wiki_46', 'wiki_47', 'wiki_48', 'wiki_49', 'wiki_50', 'wiki_51', 'wiki_52', 'wiki_53', 'wiki_54', 'wiki_55', 'wiki_56', 'wiki_57', 'wiki_58', 'wiki_59', 'wiki_60', 'wiki_61', 'wiki_62', 'wiki_63', 'wiki_64', 'wiki_65', 'wiki_66', 'wiki_67', 'wiki_68', 'wiki_69', 'wiki_70', 'wiki_71', 'wiki_72', 'wiki_73', 'wiki_74', 'wiki_75', 'wiki_76', 'wiki_77', 'wiki_78', 'wiki_79', 'wiki_80', 'wiki_81', 'wiki_82', 'wiki_83', 'wiki_84', 'wiki_85', 'wiki_86', 'wiki_87',

In [ ]:
path = '/content/gdrive/MyDrive/IE-643/text/'
paths = []
for filename in files:
  for files in files1[filename]:
    paths.append(os.path.join(path, filename, files))

len(paths) #335 total texts we have, for our task, random 1 selected
paths

['/content/gdrive/MyDrive/IE-643/text/AA/wiki_00',
 '/content/gdrive/MyDrive/IE-643/text/AA/wiki_01',
 '/content/gdrive/MyDrive/IE-643/text/AA/wiki_02',
 '/content/gdrive/MyDrive/IE-643/text/AA/wiki_03',
 '/content/gdrive/MyDrive/IE-643/text/AA/wiki_04',
 '/content/gdrive/MyDrive/IE-643/text/AA/wiki_05',
 '/content/gdrive/MyDrive/IE-643/text/AA/wiki_06',
 '/content/gdrive/MyDrive/IE-643/text/AA/wiki_07',
 '/content/gdrive/MyDrive/IE-643/text/AA/wiki_08',
 '/content/gdrive/MyDrive/IE-643/text/AA/wiki_09',
 '/content/gdrive/MyDrive/IE-643/text/AA/wiki_10',
 '/content/gdrive/MyDrive/IE-643/text/AA/wiki_11',
 '/content/gdrive/MyDrive/IE-643/text/AA/wiki_12',
 '/content/gdrive/MyDrive/IE-643/text/AA/wiki_13',
 '/content/gdrive/MyDrive/IE-643/text/AA/wiki_14',
 '/content/gdrive/MyDrive/IE-643/text/AA/wiki_15',
 '/content/gdrive/MyDrive/IE-643/text/AA/wiki_16',
 '/content/gdrive/MyDrive/IE-643/text/AA/wiki_17',
 '/content/gdrive/MyDrive/IE-643/text/AA/wiki_18',
 '/content/gdrive/MyDrive/IE-64

In [ ]:
path = '/content/gdrive/MyDrive/IE-643/text/'
from torch.utils.data import Dataset

class Build(Dataset):
  def __init__(self, paths, pun ):
    self.pundata = pun
    self.paths = paths

  def __len__(self):
      return len(self.paths)
  def __getitem__(self, idx):
      if torch.is_tensor(idx):
         idx = idx.tolist()
      
      textfile = open(paths[idx], 'r')
      text = textfile.read()
      pattern = '<[^,]+>'
      clean = re.sub(pattern , '', text)
      clean = re.sub("[0-9]", "#", clean)
      clean = re.sub("#" , "", clean)
      clean = re.sub("['']", "", clean)
      clean = re.sub("[()]", "", clean)
      clean = re.sub("=","", clean)
      clean = re.sub(":","", clean)
      clean = re.sub("%","", clean)
      clean = re.sub(";","", clean)
      clean = re.sub("-","", clean)
      clean.replace("\\", "")
      lower = clean.lower()
      split = sent_tokenize(lower)
      split2 = []
      for i in range(len(split)):
        x = word_tokenize(split[i])
        if len(x)>=5 and len(x)<= 50:
          split2.append(disambiguate(split[i]))

      
      val = []
      for value in pun.values():
        for k in range(len(value)):
          for l in range(len(value[k])):
            val.append(value[k][l])
      s = 0
      g = 0
      for k in range(len(val)):
        a = val[k][1]             
        for i in range(len(split2)):
          for j in range(len(split2[i])):#wordcapture
            g = g+1
            if str(split2[i][j][1]) == str(val[k][1]): ########Adding pseudo words to the tagged corpus
              split2[i][j] = val[k]
              s = s+1

      for i in range(len(split2)):
        j = len(split2[i])
        split2[i].insert(0, ('<sos>', None)) #marking start of sentence
        split2[i][j] = ('</s>', None) #marking end of sentence

      return split2

In [ ]:
taggedcorpus = Build(paths, pun)
import torch
%cd gdrive
%cd MyDrive
%cd IE-643
!ls

[Errno 2] No such file or directory: 'gdrive'
/content/gdrive/MyDrive/IE-643
[Errno 2] No such file or directory: 'MyDrive'
/content/gdrive/MyDrive/IE-643
[Errno 2] No such file or directory: 'IE-643'
/content/gdrive/MyDrive/IE-643
 changed.txt
 Corpusdump1.pickle
 Corpusdump2.pickle
 Corpusdump.pickle
 Corpusdump.txt
 corpus.txt
 data.json
 dump1.pickle
 dump2.pickle
 dumpfile.bz2
 dump.pickle
 enwiki-latest-pages-articles.xml.bz2
 filename.pickle
 filename.pkl
 file.pickle
 glove.6B.50d.txt
 glove.6B.zip
 glovefile
 out.csv
 out.gsheet
 out.pickle
 output0.csv
 output10.csv
 output11.csv
 output12.csv
 output13.csv
 output14.csv
 output15.csv
 output16.csv
 output17.csv
 output18.csv
 output19.csv
 output1.csv
 output2.csv
 output3.csv
 output4.csv
 output5.csv
 output6.csv
 output7.csv
 output8.csv
 output9.csv
 pundata.csv
 pundata.gsheet
 Pun-GAN
 Pun-Generation
 save.csv
 save.pickle
 save.pik
 save.txt
 semcor3.0.tar.gz
 semcor.py
 semeval2017_task7
 semeval2017_task7.tar.xz
'se

In [ ]:
import random
from random import shuffle
import numpy as np
id = np.arange(335)
random.shuffle(id)



Will take around 3 hours

In [ ]:
tagged = [] #3
for i in range(0,1): 
  tagged.append(taggedcorpus[id[i]])
#will take around 1 hour
store = tagged

In [ ]:
tot_sent = 0                 #2 corpus, # sentences
for i in range(len(tagged)):
  tot_sent += len(tagged[i])

tot_sent

6718

#Saving the corpus

#Building vocab
assign unk token to rare words

In [ ]:
#our pun words
punwords = []
for value in pun.values():
   for k in range(len(value)):
     for l in range(len(value[k])):
       punwords.append(value[k][l][0])

punwords = list(set(punwords))
len(punwords)      #(pseudo words, corr to 970 pun words)

1792

In [ ]:
#Building Vocab Target ~ 10000 words

In [ ]:
SOS_token = 1
EOS_token = 0   # End-of-sentence token
UNK_token = 2
class Vocabulary:
  # End-of-sentence token
  SOS_token = 0
  EOS_token = 1   # End-of-sentence token
  UNK_token = 2
  def __init__(self, name):
    self.name = name
    self.wordcounter = {}
    self.word2index = {"</s>":EOS_token,"<sos>": SOS_token,"<unk>":UNK_token }          #</s> end, <sos> start
    self.word2count = {"<sos>":0,"</s>": 0,"<unk>":0}
    self.index2word = {EOS_token:"</s>",SOS_token: "<sos>", UNK_token :"<unk>"}
    self.unkwords = []
    self.popqt = 0
    self.num_words = 3
    self.num_sentences = 0
    self.longest_sentence = 0

  def add_word(self, word):
    if word not in self.word2index:
    # First entry of word into vocabulary
      self.word2index[word] = self.num_words
      self.word2count[word] = 1
      self.index2word[self.num_words] = word
      self.num_words += 1
    else:
      # Word exists; increase word count
      self.word2count[word] += 1

  def remove_rare_words(self, word):
    if self.word2count[word] <= 2:
      if word not in punwords:             #removing rare words other than pun words rare words occur 3 times at the most in a small corpus
        self.unkwords.append(word)
        self.popqt +=1
        self.word2count["<unk>"] = self.popqt
        dic = self.index2word.copy()
        for key, value in dic.items():
          if value == word:
            self.index2word.pop(key)
            self.word2index.pop(value)
            self.num_words = self.num_words -1

  def to_word(self, index):

    return self.index2word[index]

  def to_index(self, word):
    return self.word2index[word]

  def wordcounter(self, word):
    return self.word2count[word] 
      
voc = Vocabulary('test')
print(voc)
len(voc.word2index)

3

In [ ]:
voc.word2index

{'</s>': 0, '<sos>': 1, '<unk>': 2}

In [ ]:
#Building vocabulary, removing rare words from vocabulary, assigning unk token
new = tagged
tagged = store
c = 0
for k in range(len(tagged)):
  for i in range(len(tagged[k])): #sentences
    for j in range(len(tagged[k][i])): #words 
      word = tagged[k][i][j][0]
      if word == "''":
        voc.unkwords.append(word)
      elif word == "``":
        voc.unkwords.append(word)
      elif word == "[":
        voc.unkwords.append(word)
      elif word == "]":
        voc.unkwords.append(word)
      else:
        voc.add_word(word)
len(voc.word2index)

16022

In [ ]:
# removing rare words from vocabulary, while simultaneously making a list to be removed from corpus
tagged = store
for k in range(len(tagged)):
  for i in range(len(tagged[k])): #sentences
    for j in range(len(tagged[k][i])): #words 
     word = tagged[k][i][j][0]
     if word == "''":
      voc.unkwords.append(word)
     elif word == "``":
       voc.unkwords.append(word)
     elif word == "[":
       voc.unkwords.append(word)
     elif word == "]":
       voc.unkwords.append(word)
     else:
       voc.remove_rare_words(word)

len(voc.word2index)       #VocabSize

6162

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
a = list(set(voc.unkwords))
print(a[1:10])
print(len(a)) #unknown words

['cathedral', 'parameters', 'arbeiterpartei/dap', 'animarum', 'presume', '/−', 'gulf', 'deputies', 'pulse']
9864


In [ ]:
corpus = tagged
len(a) #total _____ unk words in corpus
for m in range(len(corpus)):
    for j in range(len(corpus[m])):
      for k in range(len(corpus[m][j])):
        if corpus[m][j][k][0] in a:
          corpus[m][j][k] = ('<unk>', None)
 

#3min

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vocab = []
for key , value in voc.word2index.items():
  vocab.append(key)

len(vocab)

6162

In [ ]:
#Filtering corpus on basis of vocabulary
#Replace unknown words with <unk> token
#Discarding sentences with more than 2 unk tokens
#Newcorpus with labeled sentences where each word is tagged with a word sense

newsent =[]
print(len(newsent))
for m in range(len(corpus)):
    for j in range(len(corpus[m])):
      count = 0
      for k in range(len(corpus[m][j])):
        if corpus[m][j][k][0] == "<unk>":
          count = count + 1
      if count < 3:
           newsent.append(corpus[m][j])

print(len(newsent))   #41943 sentences, 20000 vocab
newsent[0]

0
4120


[('<sos>', None),
 ('floorball', None),
 ('floorball', None),
 ('is', None),
 ('a', None),
 ('type', Synset('type.n.05')),
 ('of', None),
 ('floor', Synset('floor.n.10')),
 ('hockey', Synset('ice_hockey.n.01')),
 ('with', None),
 ('five', Synset('basketball_team.n.01')),
 ('players', Synset('player.n.05')),
 ('and', None),
 ('a', None),
 ('goalkeeper', Synset('goalkeeper.n.02')),
 ('in', None),
 ('each', None),
 ('team', Synset('team.n.01')),
 ('</s>', None)]

In [ ]:
#reversed sentences for backward model

reverse=[]
newsent_rev = newsent
for i in range(len(newsent_rev)):
  reverse.append(newsent_rev[i][::-1])
reverse

[[('</s>', None),
  ('team', Synset('team.n.01')),
  ('each', None),
  ('in', None),
  ('goalkeeper', Synset('goalkeeper.n.02')),
  ('a', None),
  ('and', None),
  ('players', Synset('player.n.05')),
  ('five', Synset('basketball_team.n.01')),
  ('with', None),
  ('hockey', Synset('ice_hockey.n.01')),
  ('floor', Synset('floor.n.10')),
  ('of', None),
  ('type', Synset('type.n.05')),
  ('a', None),
  ('is', None),
  ('floorball', None),
  ('floorball', None),
  ('<sos>', None)],
 [('</s>', None),
  ('holes', Synset('hole.n.03')),
  ('with', None),
  ('testisn01', Synset('testis.n.01')),
  ('<unk>', None),
  ('a', None),
  ('and', None),
  ('sticks', Synset('stick.n.06')),
  ('with', None),
  ('<unk>', None),
  ('play', Synset('toy.v.02')),
  ('women', Synset('woman.n.02')),
  ('and', None),
  ('men', Synset('work_force.n.01')),
  ('<sos>', None)],
 [('</s>', None),
  ('periods', Synset('period.n.03')),
  ('<unk>', None),
  ('three', Synset('trey.n.02')),
  ('in', None),
  ('played', Sy

In [ ]:
#saving the corpus
import pickle
f= open("Corpusdump.pickle", "wb") 
newcorpus = []

for i in range(len(newsent)):
  for j in range(len(newsent[i])):
    newcorpus.append(newsent[i][j])

len(newcorpus)

88619

In [ ]:
with open("Corpusdump2.pickle", "wb") as f:
  pickle.dump([new[0] for new in newcorpus],f)
f.close()
with open("dump2.pickle", "wb") as f1:
  pickle.dump(voc, f1)
f1.close()

In [ ]:
import pickle

f = open("Corpusdump1.pickle", "rb")
corpusdump = pickle.load(f)

In [ ]:
f2 = open("dump1.pickle", "rb")
voc1 = pickle.load(f2)

In [ ]:
len(corpusdump)

499148

In [ ]:
len(voc)

499148

In [ ]:
#Embeddings for encoder
#Using glove
  #the -0.071549 0.093459 0.023738 -0.090339 0.056123
  #to 0.57346 0.5417 -0.23477 -0.3624 0.4037
  #and 0.20327 0.47348 0.050877 0.002103 0.060547
#embedding size - 50

import codecs
path_emb = "/content/gdrive/MyDrive/IE-643/glovefile/glove.6B.100d.txt" #File import

emb_dict = {}
emb_size = 100
with open (path_emb, 'rb') as f:
  reader = codecs.getreader('utf-8')(f)
  for line in reader:
    tokens = line.strip().split(" ")
    word = tokens[0]
    vec = list(map(float, tokens[1:]))
    emb_dict[word] = vec
    if emb_size:
      assert emb_size == len(vec), "All embedding size should be same."
    else:
      emb_size = len(vec)

In [ ]:
for key, value in voc.word2count.items():
  if key == "the":
   print(value)

10637


In [ ]:
for m in range(len(newsent)):
    for j in range(len(newsent[m])):
      token = newsent[m][j][0]
      if token not in emb_dict:
        print(token)
        emb_dict[token] = [0.0]*emb_size

for key, value in voc.word2index.items():
  token = key
  if token not in emb_dict:
    emb_dict[token] = [0.0]*emb_size

<sos>
</s>
<unk>
testisn01
rulen12
developv10
longa02
gainv08
clubn02
unihockey
shapen02
functionn03
bdivision
topographic_pointn01
staten01
goaln04
highs07
goaln01
markv05
balln06
throwv01
targetn01
spheren01
resistancen01
balln01
recordv04
timen02
matchn02
drawn03
contactn05
checkv10
perplexv01
leveln07
passv20
changen01
markeds02
hitn02
clock_timen01
contactn07
net_incomen01
matchn01
unmarrieda01
producev02
associationn04
clubn03
keepv01
celebrateds01
reservationn02
degreen02
operate_onv01
augustn01
batchn02
groupn03
mostwatched
lates04
buttocksn01
twoyear
getv25
breakv53
pricen03
yearold
prison_termn01
separatev08
priorn01
timen01
digitn01
picturen01
knockv06
issuen02
stillr04
tien09
reactv01
purposen01
occurv02
affiliationn01
takev08
fourthplaced
newcomern01
impressionn02
runnersup
run_outv04
momentumn02
straighta06
suggestionn02
chargev12
sports_fann01
eminents01
threeyear
breakv05
highlightn02
sumn05
millionayear
positionn06
servicen01
covid
weaponn01
partn10
problemn02
allseate

In [ ]:
#to initialize embedding weights

emb_mat = np.array([emb_dict[key] for key,value in voc.word2index.items() ])
emb_mat.shape

(6162, 100)

In [ ]:
emb_mat = torch.as_tensor(emb_mat,  dtype = torch.float64, device = device)
emb_mat

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0', dtype=torch.float64)

In [ ]:
pseudowords =[]
for value in pun.values():
  pseudowords.append(value[0][0][0])

len(pseudowords)

970

In [ ]:
indexes1 = []
def wordsFromcorpus(voc, cor):
  for sentence in cor:
    inde = [voc.word2index[word[0]] for word in sentence]
    inde = torch.tensor(inde, dtype=torch.long).view(-1, 1)
    indexes1.append(inde)
  return indexes1
#######################we will input indexes of words to the encoder#############

In [ ]:
c =0
for key, value in voc.word2index.items():
  voc.word2index[key] = c
  c=c+1

In [ ]:
indexes1 = []
def wordsFromcorpus(voc, cor):
  for sentence in cor:
    inde = [voc.word2index[word[0]] for word in sentence]
    inde = torch.tensor(inde, dtype=torch.long).view(-1, 1)
    indexes1.append(inde)
  return indexes1


ind = wordsFromcorpus(voc, newsent)
input_tensor = ind

input_tensor


[tensor([[ 1],
         [ 3],
         [ 3],
         [ 4],
         [ 5],
         [ 6],
         [ 7],
         [ 8],
         [ 9],
         [10],
         [11],
         [12],
         [13],
         [ 5],
         [14],
         [15],
         [16],
         [17],
         [ 0]]), tensor([[ 1],
         [18],
         [13],
         [19],
         [20],
         [ 2],
         [10],
         [22],
         [13],
         [ 5],
         [ 2],
         [24],
         [10],
         [25],
         [ 0]]), tensor([[ 1],
         [26],
         [27],
         [28],
         [15],
         [29],
         [ 2],
         [31],
         [ 0]]), tensor([[ 1],
         [ 3],
         [32],
         [33],
         [15],
         [34],
         [35],
         [36],
         [37],
         [34],
         [38],
         [39],
         [15],
         [15],
         [ 2],
         [41],
         [ 2],
         [ 0]]), tensor([[ 1],
         [43],
         [44],
         [34],
         [38],
      

In [ ]:
indexes1 = []
def wordsFromcorpus(voc, cor):
  for sentence in cor:
    inde = [voc.word2index[word[0]] for word in sentence]
    index = np.flip(np.array(inde), 0).copy()
    index = torch.from_numpy(index).view(-1,1)
    indexes1.append(index)

  return indexes1


rev = wordsFromcorpus(voc, newsent)
input_tensor_reverse = rev
rev

[tensor([[ 0],
         [17],
         [16],
         [15],
         [14],
         [ 5],
         [13],
         [12],
         [11],
         [10],
         [ 9],
         [ 8],
         [ 7],
         [ 6],
         [ 5],
         [ 4],
         [ 3],
         [ 3],
         [ 1]]), tensor([[ 0],
         [25],
         [10],
         [24],
         [ 2],
         [ 5],
         [13],
         [22],
         [10],
         [ 2],
         [20],
         [19],
         [13],
         [18],
         [ 1]]), tensor([[ 0],
         [31],
         [ 2],
         [29],
         [15],
         [28],
         [27],
         [26],
         [ 1]]), tensor([[ 0],
         [ 2],
         [41],
         [ 2],
         [15],
         [15],
         [39],
         [38],
         [34],
         [37],
         [36],
         [35],
         [34],
         [15],
         [33],
         [32],
         [ 3],
         [ 1]]), tensor([[ 0],
         [47],
         [46],
         [45],
         [36],
      

In [ ]:
type(rev[1])

torch.Tensor

In [ ]:
newsent[0]

[('<sos>', None),
 ('floorball', None),
 ('floorball', None),
 ('is', None),
 ('a', None),
 ('type', Synset('type.n.05')),
 ('of', None),
 ('floor', Synset('floor.n.10')),
 ('hockey', Synset('ice_hockey.n.01')),
 ('with', None),
 ('five', Synset('basketball_team.n.01')),
 ('players', Synset('player.n.05')),
 ('and', None),
 ('a', None),
 ('goalkeeper', Synset('goalkeeper.n.02')),
 ('in', None),
 ('each', None),
 ('team', Synset('team.n.01')),
 ('</s>', None)]

LANGUAGE MODEL (generator)

We use the data set of
homographic puns from Miller et al. (2017). The
model is trained on the data set in asynchronous
forward/backward way. As the pun data set is
limited, the pun language model has no creativity,
which means if we input a word appearing in the
training data, then the output will usually be an existing sentence from the training data. Therefore,
we remove the sentences which contain words in
the 100 target words from the pun data set, and
then train the model for test.

Model specs:
I. The conditional language model for pun generation is similar to the seq2seq model with an input of only one word. We use Long Short-Term
Memory (LSTM) as encoder to map the input sequence (target word) to a vector of a fixed dimensionality, and then another LSTM network as decoder to decode the target sequence from the vector 


II. Our goal is to generate a sentence containing the target word. However, vanilla seq2seq model cannot guarantee the target word to appear in the generated sequence all the time. To
solve this problem, we adopt the asynchronous
forward/backward generation model

III. The model first generates the
backward sequence starting from the target word
wt at position t of the sentence (i.e., the words before wt), and ending up with “</s>” at the position 0 of the sentence. 

IV.. The model first generates the
backward sequence starting from the target word
wt at position t of the sentence (i.e., the words before wt), and ending up with “</s>” at the position 0 of the sentence



In [ ]:
# First we build FORWARD ENCODER-DECODERFRAMEWORK

import torch
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as utils
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
import torch.nn.functional as F
import torchvision
from torchvision import datasets, models, transforms
from torchvision.transforms import ToTensor
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

from torchtext.data import Field
from torchtext.vocab import GloVe


PyTorch Version:  1.7.0+cu101
Torchvision Version:  0.8.1+cu101


In [ ]:
vocab = voc.word2index
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class EncoderRNN(nn.Module):                                                    #(encoder maps input to vec of fixed dimensionality-128)
    def __init__(self,emb_dim, hidden_size, vocab_size,batch_size,num_layers,  dtype = torch.float64):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(emb_dim, hidden_size )
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        self.dropout = nn.Dropout(0.2)
        self.embedding.weight.data.copy_(emb_mat)
        self.batch_size =batch_size

    def forward(self, input, hidden):

        embedded = self.embedding(input).view(1,1,-1)

        output = embedded
        output, (hidden,cell) = self.lstm(embedded, hidden )
        return output, hidden, cell

    def initHidden_(self):
        
        return (torch.rand(1, self.batch_size, self.hidden_size, device = device), torch.rand(1, self.batch_size, self.hidden_size, device = device))


In [ ]:
vocab_size = len(voc.word2index)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


batch_size = 1
encoder = EncoderRNN(100,128,vocab_size,batch_size, num_layers =1)



In [ ]:
hidden1.shape

NameError: ignored

In [ ]:
#fastr01, fastv02

for key, value in voc.word2index.items():
  if key == "fastr01":
    print(value)
  if key == "fastv02":
    print(value)

In [ ]:
#@title
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, emb_size, vocab_size, beam_width):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size,emb_size)
        self.lstm = nn.LSTM(emb_size, hidden_size)
        self.hidlin = nn.Linear(hidden_size, vocab_size)
        self.softmax = nn.LogSoftmax(dim=2)
        self.beam_width = beam_width 
        self.vocab_size = vocab_size
        self.beam_width = beam_width
        self.embedding.weight.data.copy_(emb_mat)



    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


    def Joint_Beam_Search_Decoder(self, hidden_1, hidden_2, cell1, cell2):
      
      b = self.beam_width
      Beam = {}
      U_1_in ={}
      U_2_in = {}
      
      h_state_1 = {}
      h_state_2 ={}
      c_state_1 ={}
      c_state_2 = {}
    
      for i in range(b):
        Beam[i] = []
        U_1_in[i] = torch.zeros(self.vocab_size)
        U_2_in[i] = torch.zeros(self.vocab_size)
        h_state_1[i] = hidden_1
        h_state_2[i] = hidden_2
        c_state_1[i] = cell1
        c_state_2[i] = cell2
      BeamID = torch.arange(b, dtype = torch.long) #Beam from which candidates came from
      WordID = torch.zeros(b , dtype = torch.long)  #indices of candidates in my vocabulary - Initially set to start of sentence(SOS-0)
      emb = {}
      accscore_1 = {}
      accscore_2 = {}
      joint_score ={}
      Outputs = []
      t = 0
      l=b
      while l>0:
        o = []
        for i in range(0, b):
          emb[i] = self.embedding(WordID[i]).view(1, 1, -1)
          k = h_state_1[i]
          p = h_state_2[i]
          m = c_state_1[i]
          n = c_state_2[i]
          _, (h_state_1[i], c_state_1[i]) = self.lstm(emb[i], (k, m))
          _, (h_state_2[i], c_state_2[i]) = self.lstm(emb[i], (p, n))
          print(h_state_1[i][0])
          accscore_1[i] = U_1_in[i] + (self.softmax(self.hidlin(h_state_1[i])))/ pow(30, 0.7)
          accscore_2[i] = U_2_in[i] + (self.softmax(self.hidlin(h_state_2[i]))) / pow(30, 0.7)
         
          joint_score[i] = accscore_1[i] + accscore_2[i]
          o.append(joint_score[i])
        torch.stack(o)
        O = torch.cat(o)   #b*|V|
        IDBeam = []
        scores, indices = O.squeeze(1).topk(1,1)
        WordID= indices
        BeamID= torch.arange(5)
        
        for i in range(b):
          M = Beam[BeamID[i].item()]
          M.append(WordID[i])
          Beam[i] = M
          K = O.squeeze(1)
          U1in = [K[BeamID[i]][WordID[i]]]*self.vocab_size
          U_1_in[i] = torch.FloatTensor(U1in)
          U2in = [K[BeamID[i]][WordID[i]]]*self.vocab_size
          U_2_in[i] = torch.FloatTensor(U2in)
          if WordID[i] == 1:
            l = l-1    #decrease beams which have finished searching
            Outputs.append(Beam[i])
            print(l)
        t = t+1
      return Outputs



In [ ]:
decoder = DecoderRNN(128,50,10340,5  )
Out = decoder.Joint_Beam_Search_Decoder(hid1, hid2, cell1, cell2) #Joint beam search running

Streaming output truncated to the last 5000 lines.
         -0.0268,  0.0743, -0.0790, -0.0188, -0.0396, -0.0427, -0.0044, -0.0573]],
       grad_fn=<SelectBackward>)
tensor([[-0.0039,  0.0180, -0.0741,  0.0559, -0.0486, -0.0155, -0.0059,  0.0256,
          0.0066, -0.0097,  0.0045, -0.0378, -0.0037,  0.0137, -0.0720,  0.0396,
         -0.0032, -0.0148, -0.0075,  0.0051,  0.0387,  0.0081, -0.0285, -0.0302,
          0.0553, -0.0491, -0.0114,  0.0522, -0.0141, -0.0407, -0.0103, -0.0268,
          0.0128, -0.0091, -0.0261,  0.0363,  0.0024,  0.0176,  0.0568,  0.0035,
          0.0612, -0.0120,  0.0094,  0.0406, -0.0210, -0.0359,  0.0232, -0.0429,
          0.0232,  0.0686, -0.0113, -0.0040, -0.0277, -0.0104,  0.0131,  0.0305,
         -0.0164, -0.0356,  0.0152, -0.0235, -0.0161,  0.0162,  0.0330, -0.0031,
          0.0182,  0.0035,  0.0019,  0.0549,  0.0718, -0.0207, -0.0204,  0.0147,
          0.0006, -0.0349, -0.0789, -0.0310, -0.0587, -0.0088,  0.0141,  0.0217,
         -0.0104,  0.00

KeyboardInterrupt: ignored

In [ ]:
# Modifying Decoder to use only Beam Search(instead of Joint Beam Search) in case of Training

In [ ]:
class BackwardDecoderRNN(nn.Module):
    def __init__(self, hidden_size, emb_size, vocab_size, beam_width):
        super(BackwardDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size,emb_size)
        self.lstm = nn.LSTM(emb_size, hidden_size)
        self.hidlin = nn.Linear(hidden_size, vocab_size)
        self.softmax = nn.LogSoftmax(2)
        self.beam_width = beam_width 
        self.vocab_size = vocab_size
        self.beam_width = beam_width
        self.embedding.weight.data.copy_(emb_mat)
        self.embedding = self.embedding.to(device)

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
    

    def JointBeamSearchDecoder(self, h_state_1, h_state_2, c_state_1, c_state_2, max_len,mode):
      b = self.beam_width
      Beam = {}
      U_1_in ={}
      U_2_in = {}
      for i in range(b):
        Beam[i] = []                                      #storing the generated sequence for each beam
        U_1_in[i] = torch.zeros(self.vocab_size)          #accumulated score for word1
        U_2_in[i] = torch.zeros(self.vocab_size)          #accumulated score for word2

      BeamID = torch.arange(b, dtype = torch.long) #Beam from which candidates came from (for further appending to the respective beam, to generate sequence)
      WordID = torch.ones(b, dtype = torch.long)  #indices of candidates in my vocabulary - Initially set to start of sentence(SOS-0)
      Outputs = []
      t = 0                                          #time-step
      l = b
      ID = [0,1,2,3,4]
      while l>0:
        o = []
        emb = {}
        accscore_1 = {}
        accscore_2 = {}
        joint_score ={}
        output_1 ={}
        output_2 = {}
        IDs = [0,1,2,3,4]
        if mode == "eval":
          for i in range(b):
           emb[i] = self.embedding(WordID[i]).view(1, 1, -1)
           k = h_state_1[i]
           p = h_state_2[i]
           m = c_state_1[i]
           n = c_state_2[i]
           output_1[i], (h_state_1[i], c_state_1[i]) = self.lstm(emb[i], (k, m))
           output_2[i], (h_state_2[i], c_state_2[i]) = self.lstm(emb[i], (p, n))
           accscore_1[i] = U_1_in[i] + (self.softmax(self.hidlin(output_1[i])))
           accscore_2[i] = U_2_in[i] + (self.softmax(self.hidlin(output_2[i]))) 
           joint_score[i] = accscore_1[i] + accscore_2[i] #This marks the difference between joint and single beam search            
           o.append(joint_score[i]) 
           torch.stack(o)
           O = torch.cat(o)   #b*|V|
           scores, indices = torch.topk(O.flatten(), b)
           A = np.array(np.unravel_index(indices.numpy(), O.squeeze(1).shape)).T
           WordID= torch.from_numpy(A[:,1])
           BeamID= torch.from_numpy(A[:,0])
          for i in range(b):
           M = Beam[BeamID[i].item()]
           M.append(WordID[i])
           Beam[i] = M
           K = O.squeeze(1)
           U_1_in[i] = [K[BeamID[i]][WordID[i]]]*self.vocab_size
           U_1_in[i] = torch.FloatTensor(U_1_in[i])

           U_2_in[i] = [K[BeamID[i]][WordID[i]]]*self.vocab_size
           U_2_in[i] = torch.FloatTensor(U_2_in[i])
           scores = []
           dic = {}
           if WordID[i] == 0:
             score = K[BeamID[i]][WordID[i]]
             scores.append(score)
             dic[score] = Beam[i]
             Outputs.append(dic)
             l=l-1
             b=l
             Beam[i] = []
             print(t)            
          if t>max_len:
            for i in range(b):
              score = K[BeamID[i]][WordID[i]]
              scores.append(score)
              dic[score] = Beam[i]
              Outputs.append(dic) 
              print(t)           
            return Outputs, scores
            break
            t = t+1
            return Outputs, scores
          
     ##################################################################################################################################################################################     
        if mode == "train":
           scores = []
           store_seq = []  
           u_1_in = torch.zeros([1, vocab_size], dtype = torch.float32)
           while max_len>0:
            WordID = WordID.to(device)
            emb = self.embedding(WordID).view(1, 1, -1)   
            emb = emb.to(device)
            k = h_state_1
            j = c_state_1              
            output_1, (h_state_1, c_state_1) = self.lstm(emb,(k,j))
            u_1_in = u_1_in.to(device)
            output_1 = output_1.to(device)
            predicted =  u_1_in + self.softmax(self.hidlin(output_1))   #|V| (product of probabilities for generation)
            score1 = self.softmax(self.hidlin(output_1))
            score, indice = predicted.topk(1)
            WordID = indice
            u_1_in = score
            scores.append(score1)
            store_seq.append(WordID)              
            if WordID == 0:
              break
              return store_seq, scores, u_1_in  
            max_len = max_len -1
            t = t+1
           return store_seq, scores

In [ ]:
beam_width = 5
decoder1 = BackwardDecoderRNN( 128, 100, vocab_size, 1)

In [ ]:
h_state_1 = {}
h_state_2 ={}
c_state_1 ={}
c_state_2 = {}
b = 9
for i in range(b):
  pseu1 = input_tensor[25][17]
  pseu2 = input_tensor[43][10]
  hidden = encoder.initHidden_()
  out, h_state_1[i], c_state_1[i] = encoder.forward(pseu1, hidden)
  out, h_state_2[i], c_state_2[i] = encoder.forward(pseu2, hidden)

Out1, scores1 = decoder1.JointBeamSearchDecoder( h_state_1,h_state_2, c_state_1, c_state_2, 1000,"eval")

In [ ]:
Outseq = []
scores1 = torch.tensor(scores1)
scores1 =scores1.clone().detach().requires_grad_(True)
a = torch.max(scores1)
for key, value in Out1[0].items():
  if key == a:
    Outseq.append(value)

for i in Outseq[0]:
  print(index2word[i.item()])

In [ ]:
c = []
def FinalBeamSearchOutput(Out1):
  for i in range(len(Out1)):
    c.append(torch.mode(torch.tensor(Out1[i])).values)
  k = torch.tensor(c)  
  return torch.mode(k).values

#Backward Encoder Decoder

In [ ]:
encoder_bw = EncoderRNN(100, 128, vocab_size, 1,1,  dtype = torch.float64)
hidden = encoder_bw.initHidden_()
a = input_tensor[0][9].to(device)
output, hidden1,cell = encoder_bw.forward(a,hidden)
decoder_bw = BackwardDecoderRNN(128,100,vocab_size,1)
max_len = 50
h_state_1 = hidden1
c_state_1 = cell
output, scores = decoder_bw.JointBeamSearchDecoder(h_state_1, None, c_state_1, None, max_len,mode = "train")
output

RuntimeError: ignored

In [ ]:
for i in range(len(output)):
  print(index2word[output[i].item()])

In [ ]:
output

In [ ]:
for value in Out1[0].values():
  for val in range(len(value)):
    print(index2word[val])
  print("sequence ends here")

</s>
<sos>
<unk>
acts
of
union
the
were
two
parliament
with
scotland
act
travel_byv01
by
england
,
and
in
they
sequence ends here


In [ ]:
h_state_1 = {}
h_state_2 ={}
c_state_1 ={}
c_state_2 = {}

for i in range(b):
    pseu1 = torch.as_tensor([3160])
    pseu2 = torch.as_tensor([8013])
    hidden = encoder.initHidden_()
    out, h_state_1[i], c_state_1[i] = encoder.forward(pseu1, hidden)
    out, h_state_2[i], c_state_2[i] = encoder.forward(pseu2, hidden)



Out1 = decoder.JointBeamSearchDecoder("eval", h_state_1 ,h_state_2, c_state_1, c_state_2)

In [ ]:
index2word={}
for key, value in voc.word2index.items():
  index2word[value] = key

In [ ]:
len(input_tensor_reverse[0])

24

BACKWARD MODEL

In [ ]:
x = [index2word[input_tensor[0][9].item()]]
y = [index2word[idx.item()] for idx in input_tensor_reverse[0][-9:]]
print('eg: Input:', x)
print('eg Target Output:', y)   #reverse seq

eg: Input: ['with']
eg Target Output: ['hockey', 'floor', 'of', 'type', 'a', 'is', 'floorball', 'floorball', '<sos>']


In [ ]:
#@title
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size #128
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm= nn.LSTM(emb_dim, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):  #initially, output and hidden state from encoder

        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.lstm(output, hidden)
        print(hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self, hidden_size):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
#dataset for the model

input_sents = input_tensor
target_sents = input_tensor_reverse
decoder = BackwardDecoderRNN(128,100,vocab_size, 4)
encoder = EncoderRNN(100,128,vocab_size, 1, 0)

In [ ]:
c= 0
for i in len(input_tensor_reverse):
  c = c+

22863

In [ ]:
def train(input_tensor, target_tensor, encoder, decoder, beam, encoder_optimizer, decoder_optimizer, criterion):
  encoder_optimizer.zero_grad() #
  decoder_optimizer.zero_grad() #
 
  input_length = len(input_tensor)#
  target_length = len(input_tensor)   #      #######Shifted Input tensor(to the left, to predict backward sequence)
 
  loss_ = 0
  for i in range(len(input_tensor)): #ssentence
    input_tensor[i].to(device)
    target_tensor[i] =  target_tensor[i].to(device)
    for k in range(len(input_tensor[i])): #word
      encoder_hidden = encoder.initHidden_()
      encoder_hidden[0].to(device)
      encoder_hidden[1].to(device)
      encoder_output, encoder_hidden, encoder_cell = encoder.forward(input_tensor[i][k], encoder_hidden)
      Outputs = []
      max_len = len(target_tensor[i][k:])
      Output_seq, scores = decoder.JointBeamSearchDecoder( encoder_hidden,None, encoder_cell,None,max_len, mode = "train") #teacherforcing
      b = torch.zeros([vocab_size, 1])
      for score in range(len(scores)): 
        g = torch.tensor(target_tensor[i][k:][score], dtype = torch.long)
        g = g.clone().detach()
        t = torch.tensor(score, dtype = torch.float64)
        b = torch.zeros(vocab_size, dtype = torch.long)
        b[g.item()] = 1
        loss = criterion(scores[score].squeeze(1), g )
        loss.backward(retain_graph = True)
        loss_ += loss
      loss_ = loss_/len(input_tensor[i])
    if i %100 == 0:
      print('sentence', i, 'loss', loss_)
  loss_f += loss_ 
  encoder_optimizer.step()
  decoder_optimizer.step()
  return loss_/len(input_tensor[i])

In [ ]:
total_params_encoder = sum(p.numel() for p in encoder_bw.parameters() if p.requires_grad)
print(f'{total_params_encoder:,} total parameters_encoder.')
total_decoder_params = sum(p.numel() for p in decoder_bw.parameters() if p.requires_grad)
print(f'{total_decoder_params:,} decoder parameters.')

733,960 total parameters_encoder.
1,528,858 decoder parameters.


In [ ]:
if(torch.cuda.is_available()):
   decoder_bw.cuda()# rnn is your model 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder_bw = EncoderRNN(100,128,vocab_size,1,1)
decoder_bw = BackwardDecoderRNN(128,100,vocab_size,1)
encoder_optimizer = optim.RMSprop(encoder_bw.parameters(), lr=0.1)
decoder_optimizer = optim.RMSprop(decoder_bw.parameters(), lr =0.1)
criterion = nn.NLLLoss()
encoder_bw = encoder_bw.to(device)
decoder_bw = decoder_bw.to(device)
losses = []
for epoch in range(0,2):
  loss1 = train(input_tensor_reverse, input_tensor_reverse, encoder_bw, decoder_bw, 1, encoder_optimizer, decoder_optimizer, criterion)
  losses.append(loss1)

RuntimeError: ignored

In [ ]:
def trainepochs(input_tensor, target_tensor, encoder, decoder, epoch):
  loss_total = 0  
  encoder_optimizer = optim.RMSprop(encoder.parameters(), lr=0.1)
  decoder_optimizer = optim.RMSprop(decoder.parameters(), lr =0.1)
  criterion = nn.NLLLoss()
  loss = train(input_tensor, target_tensor, encoder,decoder, 4, encoder_optimizer, decoder_optimizer, criterion)

  print('Epoch:', m, 'Loss:',  loss)
  return loss


In [ ]:
losses = []
for epoch in range(0,5):      #6000+ sentences in each epoch
  print("Epoch:",epoch)
  loss = trainepochs(input_tensor_reverse, input_tensor_reverse, encoder_bw, decoder_bw, epoch )
  losses.append(loss)

Epoch: 0


RuntimeError: ignored

In [ ]:
loss = nn.NLLLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
output.backward()
target.shape

torch.Size([3])

In [ ]:
input_tensor_reverse[i][k:]  #Example target output

tensor([[480],
        [ 17],
        [206],
        [ 11],
        [479],
        [478],
        [477],
        [ 40],
        [476],
        [ 14],
        [197],
        [475],
        [474],
        [473],
        [ 40],
        [  1]])

In [ ]:
b = torch.zeros(20, dtype = torch.long)
b[0] =

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
def evaluate(encoder, decoder, sentence, max_length
  
             :
    with torch.no_grad():
      input_tensor = pseudo_words
      encoder_hidden = encoder.initHidden()
      encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
       
    for i in range(len(input_tensor)):
      encoder_hidden = {}
      encoder_cell = {}
      decoder_input = {}
      for j in range(len(b)):
        decoder_input[j] = 1
        encoder_hidden[j] = encoder.initHidden()
        encoder_output, encoder_hidden[j], encoder_cell[i] = encoder.forward(input_tensor[i], encoder_hidden[j])

    Outputs = []
    while True: 

      
     Output, encoder_hidden, encoder_cell = decoder.BeamSearchDecoder(encoder_hidden, encoder_cell, decoder_input)
     decoder_output = FinalBeamSearchOutput(Output)
     for j in range(len(b)):
       decoder_input[j] = decoder_output.item()
       Outputs.append(decoder_output.item())
       if decoder_input.item() == SOS_token:  #Backward
          break
   
     return Outputs

SyntaxError: ignored

In [ ]:
#Evaluation ----Joint decoding is  used in evaluation

In [ ]:
torch.tensor(input_tensor_reverse[i][-k:])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


tensor([[ 0],
        [47],
        [46],
        [ 2],
        [60],
        [ 1]])

In [ ]:
import xml.etree.ElementTree as ET
import csv
tree = ET.parse('/content/gdrive/MyDrive/IE-643/Pun-Generation/sem17_task7/test/subtask3-homographic-test.xml')
root = tree.getroot()

pundata1 = open('/content/gdrive/MyDrive/IE-643/pundata.csv', 'w', newline = '', encoding = 'utf-8')
csvwriter = csv.writer(pundata1)


In [ ]:
root

<Element 'corpus' at 0x7f634db6dc78>

In [ ]:
import xml.etree.ElementTree as ET
root = tree.getroot()

# one specific item attribute


print('Item #2 attribute:')
print(root[0][1].attrib)

# all item attributes
print('\nAll attributes:')
sub = []
text = []
for elem in root:
    for subelem in elem:
      if subelem.attrib['senses'] =='2':
        sub.append(subelem.attrib)
        text.append(subelem.text)

len(text)

Item #2 attribute:
{'id': 'hom_2_2', 'senses': '1'}

All attributes:


1298

Creating the list of the pun words, and their sense keys to look up and retrieve

In [ ]:
import glob
import re
m = '0'
n ='0'
pseudo1  = '0'
pseudo2 = '0'
def get_synset_from_id(word, sense1, sense2):
  global m, n, pseudo1, pseudo2
  x = wn.synsets(word)
  for i in range(len(x)):
    lemmas = x[i].lemmas()
    for j in range(len(lemmas)):
      if lemmas[j].key() == sense1:
        m = x[i]
        pseudo1 = str(m)
        pseudo1 = re.sub('[()]', '', pseudo1)
        pseudo1 = re.sub('[.]', '', pseudo1)
        pseudo1 = re.sub('Synset', '', pseudo1)
        pseudo1 = re.sub("[']", "", pseudo1)
      if lemmas[j].key() == sense2:
        n = x[i]
        pseudo2 = str(n)
        pseudo2 = re.sub('[()]', '', pseudo2)
        pseudo2 = re.sub('[.]', '', pseudo2)
        pseudo2 = re.sub('Synset', '', pseudo2)
        pseudo2 = re.sub("[']", "", pseudo2)
  return (pseudo1, m), (pseudo2, n)



In [ ]:
 def BeamSearchDecoder(self,mode, h_state_1, c_state_1):
        b = self.beam_width
        Beam = {}
        U_1_in ={}
        for i in range(b):
          Beam[i] = []
          U_1_in[i] = torch.zeros(self.vocab_size, device = device)
          U_2_in[i] = torch.zeros(self.vocab_size, device = device)

        BeamID = torch.arange(b, dtype = torch.long) #Beam from which candidates came from
        WordID = torch.ones(b , dtype = torch.long, device= device)  #indices of candidates in my vocabulary - Initially set to start of sentence(SOS-0)
        Outputs = []
      
       t = 0      #time-step
       l=b
       while l>0:
        o = []
        emb = {}
        accscore_1 = {}
        joint_score ={}
        for i in range(0, b):
          WordId =WordID.to(device)
          emb[i] = self.embedding(WordID[i]).view(1, 1, -1)
          k = h_state_1[i].to(device)
          m = c_state_1[i].to(device)
          _, (h_state_1[i], c_state_1[i]) = self.lstm(emb[i], (k, m))
          accscore_1[i] = U_1_in[i] + (self.softmax(self.hidlin(h_state_1[i])))
          joint_score[i] = accscore_1[i] 
          o.append(joint_score[i])

        torch.stack(o)
        O = torch.cat(o)   #b*|V|
        scores, indices = O.squeeze(1).topk(1,1)
        WordID= indices
  
        BeamID= torch.arange(5)

        